# Attempt at Implementing Chemical Potentials
1. In this notebook, we have arrived at a way to calculate partial derivatives of chemical potentials 

In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sym
from scipy import optimize

pi = np.pi
Pi = sym.symbols('pi')

def sqrt(n):
    return np.sqrt(n)

def log(n):
    return np.log(n)

hc = 197.33 # MeV fm
n_sat = 0.16 # fm-3

## Declaring Classes
We declare a relativistic mean field EOS class, baryon, lepton, and meson classes. These store pertinent values for each thing. For example, each particle class stores the mass, fermi momentum, effective energy, etc for the particles.

We declare two types of classes: a numeric class that will store numerical values and a symbolic class that stores the symbols. From the latter we can arrive at a symbolic expression for the partial derivatives and stuff. Then, we can substitute in numerical values (in principle) to perform evaluations. 

### Equation of State with Coupling Constants
1. Allows for collection of coupling constants here. 
2. In the future could allow for generalization to other RMF-type models albiet at the moment, this would have to be the exact same model but with different coupling values

In [235]:
class eos:
    def __init__(self, g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c,\
                    g_sigma_LA, g_omega_LA, g_rho_LA, g_phi_LA):
        
        self.g_sigma_N = g_sigma_N
        self.g_omega_N = g_omega_N
        self.g_rho_N = g_rho_N
        self.g_phi_N = g_phi_N
        
        self.g_sigma_LA = g_sigma_LA
        self.g_omega_LA = g_omega_LA
        self.g_rho_LA = g_rho_LA
        self.g_phi_LA = g_phi_LA
        
        self.b = b
        self.c = c

In [228]:
# initializing numerical eos object 
gm3_num = eos(g_sigma_N = 8.784820, g_omega_N = 8.720086, g_rho_N = 8.544795, g_phi_N = 0.0, b = 0.008628, c = -0.002433,\
             g_sigma_LA = 5.408849, g_omega_LA = 5.813391, g_rho_LA = 0.0, g_phi_LA = -4.110688)

# initializing symbolic eos object 
    # declaring symbols 
g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c = sym.symbols('g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c')
g_sigma_LA, g_omega_LA, g_rho_LA, g_phi_LA = sym.symbols('g_sigma_Lambda, g_omega_Lamda, g_rho_Lambda, g_phi_Lambda')
gm3_sym = eos(g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c,\
                    g_sigma_LA, g_omega_LA, g_rho_LA, g_phi_LA)

### Equation of State: Symbolic

In [15]:
# declaring symbols
g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c = sym.symbols('g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c')
g_sigma_LA, g_omega_LA, g_rho_LA, g_phi_LA = sym.symbols('g_sigma_Lambda, g_omega_Lamda, g_rho_Lambda, g_phi_Lambda')

In [18]:
# initializing symbolic eos
gm3_sym = eos_sym(g_sigma_N, g_omega_N, g_rho_N, g_phi_N, b, c,\
                    g_sigma_LA, g_omega_LA, g_rho_LA, g_phi_LA)

### Sigma Field Self Interaction Term
$$
    U(\sigma) = \frac{1}{3}bm_N (g_\sigma \sigma)^3 + \frac{1}{4}c(g_\sigma \sigma)^4
$$

In [161]:
U = sym.Function('U')

### Baryons and Leptons
1. Declaring classes for baryons and leptons. Allows us to declare a baryon and lepton object that has the relevant data for each type of particle such as mass, charge, Fermi momentum, etc. 
2. We declare two types of classes: a numerical class and a symbolic class. In the future, could condense this into one class with the other type as a sub-class. The purpose of this is that we first arrive at an expression for the partial derivative of the chemical potentials symbolically. After that, we could then substitute in numerical values.

#### Declaring Numeric Classes

In [232]:
class baryon:
    def __init__(self, mass, isospin, charge):
    
        # variables to be established at baryon declaration
        self.mass = mass
        self.isospin = isospin
        self.charge = charge
    
        # variables to be stored later
        self.num_density = 0.0 
        self.frac = 0.0
        self.kf = 0.0
        self.ef = 0.0
        self.chem_pot = 0.0 

In [233]:
proton_num = baryon(939.0, 1/2, 1)
neutron_num = baryon(939.0, -1/2, 0)
lambda_num = baryon(1116.0, 0, 0)

#### Declaring Symbolic Classes

In [206]:
class lepton_sym:
    def __init__(self, mass, num_density, frac, var_type, kf, chem_pot):
        self.mass = mass
        self.num_density = num_density
        self.frac = frac
        self.var_type = var_type
        self.kf = kf
        self.chem_pot = chem_pot

In [207]:
class baryon_sym:
    def __init__(self, mass, isospin, num_density, frac, kind, var_type, mass_eff, kf, ef, chem_pot):
        self.mass = mass
        self.isospin = isospin
        self.num_density = num_density
        self.frac = frac
        self.kind = kind
        self.var_type = var_type
        
        self.mass_eff = mass_eff
        self.kf = kf
        self.ef = ef
        self.chem_pot = chem_pot

In [208]:
# initializing independent variables
nb = sym.symbols('n_B')

# electron 
me, ne, xe, kfe, mu_e = sym.symbols('m_e, n_e, x_e, k_{F_e}, mu_e')
electron_sym = lepton_sym(me, ne, xe, 'Independent', kfe, mu_e)

# lambda hyperon 
ml, nl, xl, kfl, efl, m1_eff, mu_l = sym.symbols('m_Lambda, n_Lambda, x_Lambda, k_F_Lambda, E^*_F_Lambda, m_Lambda^*, mu_Lambda')
nl = nb*xl
lambda_sym = baryon_sym(ml, 0, nl, xl, 'Hyperon', 'Independent', m1_eff, kfl, efl, mu_l)

In [213]:
# intializing dependent variables

# proton 
mp, np, xp, kfp, efp, mp_eff, mu_p = sym.symbols('m_p, n_p, x_p, k_F_p, E^*_F_p, m_p^*, mu_p')
np, xp = nb*xe, xe
proton_sym = baryon_sym(mp, 1/2, np, xp, 'Nucleon', 'Dependent', mp_eff, kfp, efp, mu_p)

# neutron 
mn, nn, xn, kfn, efn, mn_eff, mu_n = sym.symbols('m_n, n_n, x_n, k_F_n, E^*_F_n, m_n^*, mu_n')
nn, xn = nb*(1 - xp - xl), (1-xp -xl)
neutron_sym = baryon_sym(mn, -1/2, nn, xn, 'Nucleon', 'Dependent', mn_eff, kfn, efn, mu_n)

In [234]:
# Making a list of baryons
baryon_list = [proton_sym, neutron_sym, lambda_sym]
baryon_num_list = [proton_num, neutron_num, lambda_num]

### Meson Fields

In [223]:
class meson:
    def __init__(self, mass):
        self.mass = mass # in MeV
        self.field = 0.0 

class meson_sym:
    def __init__(self, mass, field):
        self.mass = mass
        self.field = field

In [224]:
sigma = meson_sym(sym.symbols('m_sigma'), sym.symbols('sigma'))
omega = meson_sym(sym.symbols('m_omega'), sym.symbols('omega'))
rho = meson_sym(sym.symbols('m_rho'), sym.symbols('rho'))
phi = meson_sym(sym.symbols('m_phi'), sym.symbols('phi'))

sigma_num = meson(550.0)
omega_num = meson(783.0)
rho_num = meson(770.0)
phi_num = meson(1020.0)

### Establishing Helpful Lists
1. Below we have two lists: a list of the independent variables and a list of the baryons. We sum over these in the code below.

In [181]:
independent_vars_list = [nb, xe, xl]
baryon_list = [neutron_sym, proton_sym, lambda_sym]

## Lepton/Electron Chemical Potential Derivative
$$
    \mu_e = \sqrt{k_{F_e}^2 + m_e^2} \qquad k_{F_e} = (3\pi^2 n_e)^{1/3} \qquad 
       n_e = n_B x_e
$$

In [217]:
def chem_pot_electron(x_j):
    mu_e = sym.sqrt(electron_sym.kf**2 + electron_sym.mass**2)
    mu_e = mu_e.subs(electron_sym.kf, (3*Pi**2*sym.symbols('n_B')*electron_sym.frac)**(sym.S(1)/3))
    return mu_e.diff(x_j)

# Working towards Baryon Chemical Potential Partial Derivatives
Ultimately, we want to write down a function that takes a baryon $i$ and an independent variable $x_j$ and returns the partial derivative of the chemical potential of baryon $i$ with respect to $x_j$. That is,
$$
    \frac{\partial \mu_i}{\partial x_j} = \frac{\partial \mu_i'}{\partial x_j} + \frac{\partial \mu_i^R}{\partial x_j}
$$
where 
$$
    \frac{\partial \mu_i'}{\partial x_j} = \frac{\partial }{\partial x_j}\sqrt{k_{F_i}^2 + {m_i^*}^2}
    \qquad 
    \frac{\partial \mu_i^R}{\partial x_j} = \sum_k g_{\text{Meson}}\frac{\partial }{\partial x_j}\text{Meson}_j
$$
This task is broken down as follows:
- Calculate $\partial \mu_i'/\partial x_j$ and $\partial \mu_i^R/\partial x_j$
    - For the second task, we need to calculate partial derivatives of meson fields and then multiply by relevant coupling constants 
    - For the first task, then need to find
    
With this in mind, let us start from the bottom and work ourselves back up.

## Working Towards $\partial \mu^R_i/\partial x_j$

#### Calculating Partial Derivatives of Meson Fields with respect to independent variable $x_j$
1. Goal here: write a function that takes as input independent variable $x_j$ and returns $\partial \omega_0/\partial x_j$ and more.
2. At the moment, we have three functions. But in principle, they are all quite similar. Could try to generalize this by associating to each meson class object an 'Equation of state' function as well as their relevant coupling constants. Would allow us to condense these three equations into one single equation.

In [96]:
def partial_omega(x_j):
    # returns domega/dxj
    omega.field = 0
    
    # equation of motion 
    for i in range(len(baryon_list)):
        if(baryon_list[i].kind == 'Nucleon'):
            omega.field = gm3_sym.g_omega_N*baryon_list[i].num_density + omega.field
        elif(baryon_list[i].kind == 'Hyperon'):
            omega.field = gm3_sym.g_omega_LA*baryon_list[i].num_density + omega.field
        
    omega.field = 1/omega.mass**2*omega.field 
    
    # calculate partial derivative 
    return sym.simplify(omega.field.diff(x_j))

In [105]:
def partial_rho(x_j):
    # returns drho/dxj
    rho.field = 0 
    
    # equation of motion
    # in the future would be good to call the equation of motion directly here... from Lagrangian
    for i in range(len(baryon_list)):
        if(baryon_list[i].kind == 'Nucleon'):
            rho.field = gm3_sym.g_rho_N*baryon_list[i].num_density*baryon_list[i].isospin + rho.field
        elif(baryon_list[i].kind == 'Hyperon'):
            rho.field = gm3_sym.g_rho_LA*baryon_list[i].num_density*baryon_list[i].isospin + rho.field
    
    rho.field = 1/rho.mass**2*rho.field
    
    # calculate partial derivative
    return sym.simplify(rho.field.diff(x_j))

In [117]:
def partial_phi(x_j):
    # returns dmeson/dxj
    phi.field = 0
    
    # equation of motion 
    for i in range(len(baryon_list)):
        if(baryon_list[i].kind == 'Nucleon'):
            phi.field = gm3_sym.g_phi_N*baryon_list[i].num_density + phi.field
        elif(baryon_list[i].kind == 'Hyperon'):
            phi.field = gm3_sym.g_phi_LA*baryon_list[i].num_density + phi.field
        
    phi.field = 1/phi.mass**2*phi.field 
    
    # calculate partial derivative 
    return sym.simplify(phi.field.diff(x_j))

### Arriving at expression for Partial Derivative of $\mu_i^R$
Where 
$$
    \mu_i^R = g_{\omega i}\omega + g_{\phi i}\phi + I_{3B}g_{\rho i}\rho
$$

In [115]:
def partial_mu_R(baryon, x_j):
    # returns dmu_i^R/dx_j
    
    if(baryon.kind == 'Nucleon'):
        return gm3_sym.g_omega_N*partial_omega(x_j) + gm3_sym.g_phi_N*partial_phi(x_j)\
            + baryon.isospin*gm3_sym.g_rho_N*partial_rho(x_j)
    
    elif(baryon.kind == 'Hyperon'):
        return gm3_sym.g_omega_LA*partial_omega(x_j) + gm3_sym.g_phi_LA*partial_omega(x_j)\
            + baryon.isospin*gm3_sym.g_rho_LA*partial_rho(x_j)

## Working towards $\partial \mu_i'/\partial x_j$
We have the following expression for 
$$
    \frac{\partial \mu_i'}{\partial x_j} = \frac{\partial }{\partial x_j}\sqrt{k_{F_i}^2 + {m_i^*}^2}
    = \frac{1}{2}\frac{k_{F_i}\dfrac{\partial k_{F_i}}{\partial x_j} - g_{\sigma i}m_i^* \dfrac{\partial\sigma}{\partial x_j}}{\sqrt{k_{F_i}^2 + {m_i^*}^2}}
$$
which depends on 
$$
    \frac{\partial k_{F_i}}{\partial x_j} \qquad \frac{\partial\sigma}{\partial x_j}
$$

#### Calculating $\frac{\partial k_{F_i}}{\partial x_j}$ with respect to independent variables

In [188]:
def partial_fermi(baryon, x_j):
    # assumes baryon number density has already been re-written in terms of independent variables at the beginning
    kFi = (3*Pi**2*baryon.num_density)**(sym.S(1)/3)
    return kFi.diff(x_j)

### Calculate $\partial \sigma/\partial x_j$
From the notes, we have
$$
    \frac{\partial \sigma}{\partial x_j} = \frac{\sum_i g_{\sigma i} \beta_i \dfrac{\partial k_{F_i}}{\partial x_j}}{m_\sigma^2 + \dfrac{\partial^2 U}{\partial \sigma^2} - \sum_i g_{\sigma i}\alpha_i}
$$
where
$$
    \alpha_i = \left[\frac{3}{2}\frac{g_{\sigma i}{m_i^*}^2}{\pi^2}
        \ln\frac{k_{F_i} + E_{F_i}}{m_i^*} -\frac{g_{\sigma i}}{\pi^2}\left(\frac{1}{2}k_{F_i}E_{F_i} + {m_i^*}^2\frac{k_{F_i}}{E_{F_i}}\right)\right]
$$
and
$$
    \beta_i = \frac{m_i^*}{\pi^2}\frac{k_{F_i}^2}{E_{F_i}}
$$

We have alpha given here but as we see, the code is a little ugly and redundant. We can improve this by using a sub class hopefully? Not esssential I guess.

In [142]:
def alpha(baryon):
    if (baryon.kind == 'Nucleon'):
        term1 = (3/2/Pi**2)*gm3_sym.g_sigma_N*baryon.mass_eff**2*sym.log((baryon.kf + baryon.ef)/baryon.mass_eff)
        term2 = (1/2)*baryon.kf*baryon.ef 
        term3 = baryon.mass_eff**2*baryon.kf/baryon.ef
        return term1 - gm3_sym.g_sigma_N/Pi**2*(term2 + term3)
    elif (baryon.kind == 'Hyperon'):
        term1 = (3/2/Pi**2)*gm3_sym.g_sigma_LA*baryon.mass_eff**2*sym.log((baryon.kf + baryon.ef)/baryon.mass_eff)
        term2 = (1/2)*baryon.kf*baryon.ef 
        term3 = baryon.mass_eff**2*baryon.kf/baryon.ef
        return term1 - gm3_sym.g_sigma_LA/Pi**2*(term2 + term3)

def beta(baryon):
    return baryon.mass_eff*baryon.kf**2/Pi**2/baryon.ef

Now that we have our two ''unknowns'' we can plug back into the specified expression for partial sigma.

In [170]:
def partial_sigma(baryon, x_j):
    # returns dsigma/dx_j
    
    numerator = 0 
    denominator = sigma.mass**2 + sym.diff(U(sym.symbols('sigma')),sym.symbols('sigma'),sym.symbols('sigma'))
    for i in range(len(baryon_list)):
        if (baryon_list[i].kind == 'Nucleon'):
            numerator = numerator + gm3_sym.g_sigma_N*beta(baryon_list[i])*partial_fermi(baryon, x_j)
            denominator = denominator - gm3_sym.g_sigma_N*alpha(baryon_list[i])
        elif (baryon_list[i].kind == 'Hyperon'):
            numerator = numerator + gm3_sym.g_sigma_LA*beta(baryon_list[i])*partial_fermi(baryon, x_j)
            denominator = denominator - gm3_sym.g_sigma_LA*alpha(baryon_list[i])
    
    return numerator/denominator

### Arriving at expression for $\partial \mu'_i/\partial x_j$
With both $    \frac{\partial k_{F_i}}{\partial x_j}$ and $\frac{\partial\sigma}{\partial x_j}$ in hand, we have 
$\partial \mu'_i/\partial x_j$ via 
$$
    \frac{\partial \mu_i'}{\partial x_j} = \frac{\partial }{\partial x_j}\sqrt{k_{F_i}^2 + {m_i^*}^2}
    = \frac{1}{2}\frac{k_{F_i}\dfrac{\partial k_{F_i}}{\partial x_j} - g_{\sigma i}m_i^* \dfrac{\partial\sigma}{\partial x_j}}{\sqrt{k_{F_i}^2 + {m_i^*}^2}}
$$

In [183]:
def partial_mu_prime(baryon, x_j):
    if (baryon.kind == 'Nucleon'):
        return sym.S(1)/2/sym.sqrt(baryon.kf**2 + baryon.mass_eff**2)*(baryon.kf*partial_fermi(baryon, x_j)\
                - gm3_sym.g_sigma_N*baryon.mass_eff*partial_sigma(baryon, x_j))
    elif (baryon.kind == 'Hyperon'):
        return sym.S(1)/2/sym.sqrt(baryon.kf**2 + baryon.mass_eff**2)*(baryon.kf*partial_fermi(baryon, x_j)\
                - gm3_sym.g_sigma_LA*baryon.mass_eff*partial_sigma(baryon, x_j))

## Adding $\partial \mu'_i/\partial x_j$ and $\partial \mu^R_i/\partial x_j$ together to get $\partial \mu_i/\partial x_j$
From $$
    \frac{\partial \mu_i}{\partial x_j} = \frac{\partial \mu_i'}{\partial x_j} + \frac{\partial \mu_i^R}{\partial x_j}
$$


In [190]:
def chem_pot_part_deriv(baryon, x_j):
    return partial_mu_prime(baryon, x_j) + partial_mu_R(baryon, x_j)

In [192]:
chem_pot_part_deriv(proton_sym, xe)

0.5*g_rho_N**2*n_B/m_rho**2 + (-g_sigma_N*m_p^**(3**(1/3)*g_sigma_N*k_F_p**2*m_p^**(n_B*pi**2*x_e)**(1/3)/(3*E^*_F_p*pi**2*x_e) + 3**(1/3)*g_sigma_N*k_F_n**2*m_n^**(n_B*pi**2*x_e)**(1/3)/(3*E^*_F_n*pi**2*x_e) + 3**(1/3)*g_sigma_Lambda*k_F_Lambda**2*m_Lambda^**(n_B*pi**2*x_e)**(1/3)/(3*E^*_F_Lambda*pi**2*x_e))/(-g_sigma_Lambda*(1.5*g_sigma_Lambda*m_Lambda^***2*log((E^*_F_Lambda + k_F_Lambda)/m_Lambda^*)/pi**2 - g_sigma_Lambda*(0.5*E^*_F_Lambda*k_F_Lambda + k_F_Lambda*m_Lambda^***2/E^*_F_Lambda)/pi**2) - g_sigma_N*(1.5*g_sigma_N*m_n^***2*log((E^*_F_n + k_F_n)/m_n^*)/pi**2 - g_sigma_N*(0.5*E^*_F_n*k_F_n + k_F_n*m_n^***2/E^*_F_n)/pi**2) - g_sigma_N*(1.5*g_sigma_N*m_p^***2*log((E^*_F_p + k_F_p)/m_p^*)/pi**2 - g_sigma_N*(0.5*E^*_F_p*k_F_p + k_F_p*m_p^***2/E^*_F_p)/pi**2) + m_sigma**2 + Derivative(U(sigma), (sigma, 2))) + 3**(1/3)*k_F_p*(n_B*pi**2*x_e)**(1/3)/(3*x_e))/(2*sqrt(k_F_p**2 + m_p^***2))

## Numerical Calculations
1. We have a symbolic expression for the chemical potential partial derivatives. In this part, we want to then get the numerical expressions. We substitute in for all variables the values of the fields, masses, coupling constants, fractions, etc. The plan here is to take the data from the data file and for each data row (ie, for a given nB and other fixed fractions and stuff) and store that in the numeric baryon class. Assuming that is done, we can then...

In [240]:
def chem_pot_part_deriv_num(baryon, x_j):
    symbolic_part_deriv = chem_pot_part_deriv(baryon, x_j)
    
    # replace coupling constants
    symbolic_part_deriv = symbolic_part_deriv.subs([(gm3_sym.g_sigma_N, gm3.g_sigma_N),\
                                (gm3_sym.g_sigma_LA, gm3.g_sigma_LA), (gm3_sym.g_omega_N, gm3.g_omega_N),\
                                (gm3_sym.g_omega_LA, gm3.g_omega_LA), (gm3_sym.g_phi_N, gm3.g_phi_N),\
                                (gm3_sym.g_phi_LA, gm3.g_phi_LA), (gm3_sym.g_rho_N, gm3.g_rho_N),\
                                (gm3_sym.g_rho_LA, gm3.g_rho_LA)])

    # replace meson field masses
    symbolic_part_deriv = symbolic_part_deriv.subs([(omega.mass, omega_num.mass), (sigma.mass, sigma_num.mass),\
                                                   (phi.mass, phi_num.mass), (rho.mass, rho_num.mass)])
    
    # replace effective masses
    for baryon in baryon_list:
        if (baryon.kind == "Nucleon"):
            symbolic_part_deriv = symbolic_part_deriv.subs(baryon.mass_eff, baryon.mass - gm3.g_omega_N*sigma.field)
        elif (baryon.kind == "Hyperon"):
            symbolic_part_deriv = symbolic_part_deriv.subs(baryon.mass_eff, baryon.mass - gm3.g_omega_LA*sigma.field)
    
    # replace fermi momentum
    for i in range(len(baryon_list)):
        symbolic_part_deriv = symbolic_part_deriv.subs(baryon_list[i].kf, baryon_num_list[i].kf)
    
    # replace effective energy
    for i in range(len(baryon_list)):
        symbolic_part_deriv = symbolic_part_deriv.subs(baryon_list[i].ef, baryon_num_list[i].ef)
    
    # replace baryon number density
    for i in range(len(baryon_list)):
        symbolic_part_deriv = symbolic_part_deriv.subs(baryon_list[i].num_density, baryon_num_list[i].num_density)
    
    # replace particle fractions 
    for i in range(len(baryon_list)):
        symbolic_part_deriv = symbolic_part_deriv.subs(baryon_list[i].frac, baryon_num_list[i].frac)
    for i in range(len(lepton_list)):
        symbolic_part_deriv = symbolic_part_deriv.subs(lepton_list[i].frac, lepton_num_list[i].frac)
    
    # replace partial derivative of U self energy
    symbolic_part_deriv = symbolic_part_deriv.subs(sym.diff(U(sym.symbols('sigma')),sym.symbols('sigma'),sym.symbols('sigma')),\
                                                  2*gm3.b*gm3.g_sigma_N**3*sigma.field + 3*gm3.c*gm3.g_sigma_N**4*sigma.field**2)
    # evaluate 
    return symbolic_part_deriv

In [241]:
chem_pot_part_deriv_num(proton_sym, nb)

AttributeError: 'baryon' object has no attribute 'ef'

## Loading in Data
1. We read in the data.

### Tilde

In [215]:
mu_xe = neutron_sym.chem_pot - proton_sym.chem_pot - electron_sym.chem_pot
mu_xl = neutron_sym.chem_pot - lambda_sym.chem_pot

In [212]:
neutron_sym.chem_pot

mu_p

## Future Considerations
1. Take what we've done here and write these functions as library functions... Would be amazing if could just call a function that immediately calculates the partial derivatives.